This notebook presents different chunking algorithms that claim to provide _optimal_ access.

# Import section

In [2]:
import sys
import os.path as osp
osp.curdir
# Modify system path to import needed modules
sys.path.append(osp.abspath(osp.join(osp.curdir,
                                     osp.pardir, 'lib')))
from chunking import unidata_chunk, h5py_chunk

# Chunking of CoRTAD Data

CoRTAD datasets' shape is either (time=1617, lat=4320, lon=8640) or (lat=4320, lon=8640). The datasets are chunked as (101, 540, 540) or (540, 540), respectively. Dataset datatype is either `H5T_STD_I16LE` or `H5T_STD_I8LE` (datasets are scaled).

The datasets are compressed with `H5Z_FILTER_DEFLATE`, level 1.

## Unidata formula

In [6]:
unidata_chunk((1617, 4320, 8640), valSize=2)

(4, 235, 470)

In [8]:
unidata_chunk((1617, 4320, 8640), valSize=2, chunkSize=4096)

(1, 32, 64)

In [9]:
unidata_chunk((1617, 4320, 8640), valSize=1)

(6, 279, 558)

In [10]:
unidata_chunk((1617, 4320, 8640), valSize=1, chunkSize=4096)

(1, 45, 91)

## h5py formula

In [11]:
h5py_chunk((1617, 4320, 8640), typesize=2)

(26, 68, 135)

In [12]:
h5py_chunk((1617, 4320, 8640), typesize=2, chunk_size=4096)

(4, 9, 34)

In [13]:
h5py_chunk((1617, 4320, 8640), typesize=1)

(26, 68, 270)

In [14]:
h5py_chunk((1617, 4320, 8640), typesize=1, chunk_size=4096)

(7, 17, 34)

# Testing area below...

In [5]:
unidata_chunk([9812, 720, 180], chunkSize=4096)

(8, 21, 6)

In [4]:
h5py_chunk([9812, 720, 180])

(307, 23, 6)